# Quickstart

## Basic Example

First, let's start with a basic example, to see how easy it is to switch between providers

In [1]:
from v_router import Client, LLM, BackupModel

llm_config = LLM(
        model_name="claude-sonnet-4",
        provider="anthropic",
        max_tokens=100,
        temperature=0
    )

client = Client(llm_config)

response = await client.messages.create(
    messages=[
        {"role": "user", "content": "Say hello in one sentence."}
    ]
)

print(f"Response: {response.content}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")

2025-05-29 18:46:19,171 - v_router.router - INFO - Trying primary model: claude-sonnet-4 on anthropic


Response: Hello, it's nice to meet you!
Model: claude-sonnet-4-20250514
Provider: anthropic


## Fallback Example
Here we see what happens when you set a model fallback, if the main model fails VRouter will try the fallback models 

In [2]:
llm_config = LLM(
        model_name="claude-6",  # Primary model
        provider="anthropic",
        max_tokens=100,
        backup_models=[
            BackupModel(
                model=LLM(
                    model_name="gpt-4o",
                    provider="openai"
                ),
                priority=1  # First fallback
            ),
            BackupModel(
                model=LLM(
                    model_name="gemini-1.5-pro",
                    provider="google"
                ),
                priority=2  # Second fallback
            )
        ]
    )
client = Client(llm_config)

# This will try claude-3-opus first, then gpt-4, then gemini-1.5-pro
response = await client.messages.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What's 2+2?"}
        ]
    )
    
print(f"Response: {response.content}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")


2025-05-29 18:46:20,845 - v_router.router - INFO - Trying primary model: claude-6 on anthropic
2025-05-29 18:46:21,067 - v_router.router - WARNING - Primary model failed: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-6'}}
2025-05-29 18:46:21,068 - v_router.router - INFO - Trying backup model: gpt-4o on openai


Response: 2 + 2 equals 4.
Model: gpt-4o-2024-08-06
Provider: openai


## Cross Provider Switch
You can set try other providers = True.
If a call fails the system will try to call another provider with the same model

In [3]:
llm_config = LLM(
        model_name="claude-opus-4",
        provider="vertexai",  # Try Anthropic first
        max_tokens=100,
        try_other_providers=True  # Enable cross-provider fallback
    )

client = Client(llm_config)

response = await client.messages.create(
        messages=[
            {"role": "user", "content": "Tell me a short joke."}
        ]
    )
    
print(f"Response: {response.content}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")

2025-05-29 18:46:22,878 - v_router.router - INFO - Trying primary model: claude-opus-4 on vertexai
2025-05-29 18:46:23,770 - v_router.router - WARNING - Primary model failed: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.
2025-05-29 18:46:23,770 - v_router.router - INFO - Trying alternative provider: claude-opus-4 on anthropic


Response: Why don't scientists trust atoms?

Because they make up everything!
Model: claude-opus-4-20250514
Provider: anthropic


# Function Calling / Tools

V-Router supports function calling across all providers! You can define tools once and they'll work with Anthropic, OpenAI, Google, and Azure models.

## Basic Function Calling

Let's start with a simple weather tool example.

In [ ]:
from pydantic import BaseModel, Field
from v_router.classes.tools import ToolCall, Tools

# Define the tool schema using Pydantic
class WeatherQuery(BaseModel):
    """Schema for weather query parameters."""
    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")
    units: str = Field("fahrenheit", description="Temperature units: fahrenheit or celsius")

# Create a tool definition
weather_tool = ToolCall(
    name="get_weather",
    description="Get the current weather in a given location",
    input_schema=WeatherQuery.model_json_schema()
)

# Create tools collection
tools = Tools(tools=[weather_tool])

# Create LLM configuration with tools
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=tools  # Pass the tools to the LLM
)

client = Client(llm_config)

# Make a request that should trigger the tool
response = await client.messages.create(
    messages=[
        {"role": "user", "content": "What's the weather like in San Francisco?"}
    ]
)

print(f"Response type: {type(response.content)}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")

# Check if the response contains tool calls
if isinstance(response.content, list):
    print("\nContent blocks:")
    for i, block in enumerate(response.content):
        print(f"  Block {i}: {block}")
        if hasattr(block, 'type'):
            print(f"    Type: {block.type}")
            if block.type == 'tool_use':
                print(f"    Tool: {block.name}")
                print(f"    Input: {block.input}")
else:
    print(f"\nText response: {response.content}")

## Multiple Tools Example

You can define multiple tools for more complex workflows:

In [ ]:
# Define multiple tool schemas
class CalculatorQuery(BaseModel):
    """Schema for calculator operations."""
    operation: str = Field(..., description="The mathematical operation: add, subtract, multiply, divide")
    a: float = Field(..., description="First number")
    b: float = Field(..., description="Second number")

class TimeQuery(BaseModel):
    """Schema for time queries."""
    timezone: str = Field("UTC", description="Timezone (e.g., UTC, America/New_York, Europe/London)")

# Create multiple tools
calculator_tool = ToolCall(
    name="calculator",
    description="Perform basic mathematical operations",
    input_schema=CalculatorQuery.model_json_schema()
)

time_tool = ToolCall(
    name="get_current_time",
    description="Get the current time in a specified timezone",
    input_schema=TimeQuery.model_json_schema()
)

weather_tool = ToolCall(
    name="get_weather",
    description="Get the current weather in a given location",
    input_schema=WeatherQuery.model_json_schema()
)

# Create tools collection with multiple tools
multi_tools = Tools(tools=[calculator_tool, time_tool, weather_tool])

# Create LLM configuration with multiple tools
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=multi_tools
)

client = Client(llm_config)

# Test with a query that could use multiple tools
response = await client.messages.create(
    messages=[
        {"role": "user", "content": "What's 15 * 23? Also, what time is it in New York?"}
    ]
)

print(f"Response: {response.content}")
print(f"Raw tool calls: {[content for content in response.raw_response.content if hasattr(content, 'type') and content.type == 'tool_use']}")

## Cross-Provider Function Calling

The same tools work across different providers! Let's test with OpenAI:

In [ ]:
# Same tools, different provider
llm_config_openai = LLM(
    model_name="gpt-4",
    provider="openai",
    tools=multi_tools  # Same tools work across providers!
)

client_openai = Client(llm_config_openai)

response_openai = await client_openai.messages.create(
    messages=[
        {"role": "user", "content": "Calculate 42 divided by 7"}
    ]
)

print("=== OpenAI Provider ===")
print(f"Response type: {type(response_openai.content)}")
print(f"Provider: {response_openai.provider}")
print(f"Model: {response_openai.model}")

# OpenAI returns a ChatCompletionMessage object when tools are called
if hasattr(response_openai.content, 'tool_calls') and response_openai.content.tool_calls:
    print("\nTool calls detected:")
    for tool_call in response_openai.content.tool_calls:
        print(f"  Tool: {tool_call.function.name}")
        print(f"  Arguments: {tool_call.function.arguments}")
else:
    print(f"Text: {response_openai.content}")

# Let's also try with Google
llm_config_google = LLM(
    model_name="gemini-1.5-pro",
    provider="google",
    tools=Tools(tools=[calculator_tool])  # Just the calculator for Google
)

client_google = Client(llm_config_google)

response_google = await client_google.messages.create(
    messages=[
        {"role": "user", "content": "What is 8 times 9?"}
    ]
)

print("\n=== Google Provider ===")
print(f"Response type: {type(response_google.content)}")
print(f"Provider: {response_google.provider}")
print(f"Model: {response_google.model}")

# Google returns a list of Part objects when tools are called
if isinstance(response_google.content, list):
    for part in response_google.content:
        if hasattr(part, 'function_call') and part.function_call:
            print("\nFunction call detected:")
            print(f"  Function: {part.function_call.name}")
            print(f"  Arguments: {part.function_call.args}")
else:
    print(f"Text: {response_google.content}")

## Function Calling with Fallbacks

Tools work seamlessly with the fallback system. If the primary model fails, the backup models will inherit the same tools:

In [ ]:
# Create a configuration with a non-existent primary model and fallbacks
llm_config_with_fallback = LLM(
    model_name="claude-nonexistent",  # This will fail
    provider="anthropic",
    tools=multi_tools,  # Tools will be passed to fallback models too
    backup_models=[
        BackupModel(
            model=LLM(
                model_name="gpt-4",
                provider="openai"
                # Note: No tools specified here - they'll be inherited from primary
            ),
            priority=1
        ),
        BackupModel(
            model=LLM(
                model_name="gemini-1.5-pro", 
                provider="google"
                # Note: No tools specified here - they'll be inherited from primary
            ),
            priority=2
        )
    ]
)

client_fallback = Client(llm_config_with_fallback)

# This will fail on the primary model but succeed on the fallback with tools intact
response_fallback = await client_fallback.messages.create(
    messages=[
        {"role": "user", "content": "Please calculate 100 divided by 5"}
    ]
)

print(f"Response type: {type(response_fallback.content)}")
print(f"Provider used: {response_fallback.provider}")
print(f"Model used: {response_fallback.model}")

# Check if tools were used
if hasattr(response_fallback.content, 'tool_calls') and response_fallback.content.tool_calls:
    print("\n✅ Tools work seamlessly with fallbacks!")
    print("Tool calls:")
    for tool_call in response_fallback.content.tool_calls:
        print(f"  Tool: {tool_call.function.name}")
        print(f"  Arguments: {tool_call.function.arguments}")
elif isinstance(response_fallback.content, str):
    print(f"\nText response: {response_fallback.content}")
else:
    print(f"\nRaw content: {response_fallback.content}")

## Handling Tool Responses

In a real application, you'd want to execute the tools and provide results back to the model. Here's a complete example:

In [ ]:
import json
from datetime import datetime

def execute_tool(tool_name: str, tool_input: dict) -> str:
    """Simulate executing tools and returning results."""
    if tool_name == "calculator":
        operation = tool_input["operation"]
        a = tool_input["a"]
        b = tool_input["b"]
        
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            result = a / b if b != 0 else "Error: Division by zero"
        else:
            result = "Error: Unknown operation"
            
        return f"The result of {a} {operation} {b} is {result}"
    
    elif tool_name == "get_current_time":
        # Simulate getting current time
        return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (simulated)"
    
    elif tool_name == "get_weather":
        location = tool_input["location"]
        units = tool_input.get("units", "fahrenheit")
        # Simulate weather data
        return f"The weather in {location} is 72°F (22°C), sunny with light winds (simulated)"
    
    return "Tool execution not implemented"

# Create a simple calculator tool for this example
simple_calc_tool = ToolCall(
    name="calculator", 
    description="Perform basic math operations",
    input_schema=CalculatorQuery.model_json_schema()
)

llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=Tools(tools=[simple_calc_tool])
)

client = Client(llm_config)

# Initial request
messages = [
    {"role": "user", "content": "What is 25 times 4?"}
]

response = await client.messages.create(messages=messages)

print("Initial response:")
print(f"Content: {response.content}")

# Check if there are tool calls in the response
tool_calls = [content for content in response.raw_response.content if hasattr(content, 'type') and content.type == 'tool_use']

if tool_calls:
    print(f"\nFound {len(tool_calls)} tool call(s):")
    
    # Add the assistant's response to conversation
    messages.append({
        "role": "assistant", 
        "content": response.raw_response.content
    })
    
    # Execute each tool and add results
    for tool_call in tool_calls:
        print(f"Executing tool: {tool_call.name}")
        print(f"Input: {tool_call.input}")
        
        # Execute the tool
        result = execute_tool(tool_call.name, tool_call.input)
        print(f"Result: {result}")
        
        # Add tool result to conversation
        messages.append({
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_call.id,
                    "content": result
                }
            ]
        })
    
    # Get final response with tool results
    final_response = await client.messages.create(messages=messages)
    print(f"\nFinal response with tool results:")
    print(f"Content: {final_response.content}")
else:
    print("No tool calls found in response")

## Summary

✅ **Function calling works across all providers**: Anthropic, OpenAI, Google, and Azure
✅ **Unified interface**: Define tools once, use everywhere  
✅ **Fallback support**: Tools are automatically passed to backup models
✅ **Type safety**: Use Pydantic schemas for tool parameters
✅ **Cross-provider compatibility**: Same tool definitions work with different providers

The v-router package now provides a truly unified interface for function calling across all major LLM providers!